In [349]:
import numpy as np
import pandas as pd
import csv
import re
import math

### Preprocessing

In [350]:
#patterns for all the other unicodes except for korean,english,and number
en_ko_pattern = re.compile("["
        u"\U00000021-\U0000002F"
        u"\U0000003A-\U00000040"
        u"\U0000005B-\U00000060"
        u"\U0000007B-\U000010ff"
        u"\U00001113-\U00001160"
        u"\U00001176-\U000011A7"
        u"\U000011C3-\U00003130"
        u"\U0000314f-\U0000ABFF"
        u"\U0000D7A4-\U0010FFFF"
                           "]+", flags=re.UNICODE)

In [351]:
#replace meaningless unicodes, hash, and html tags
insta_contents = []
with open('jeju.csv','r') as f:
    csv_reader = csv.reader(f)
    next(csv_reader)
    for line in csv_reader:
        content = line[2].replace('#',' ').replace('&nbsp;',' ').replace('&gt;',' ').replace('&lt;',' ')
        content = en_ko_pattern.sub(r' ',content)
        insta_contents.append(content)

In [352]:
#remove duplicates
insta_contents = list(dict.fromkeys(insta_contents))

In [353]:
with open('jeju_contents.csv','w') as f:
    for content in insta_contents:
        f.write(content+'\n')

### Tagging
따로 광고인지 아닌지 태그가 되어있지 않은 데이터이기 때문에 트레이닝을 위해서 수작업으로 태깅을 했다.

In [354]:
ads, non_ads, semi_ads, reviews = [],[],[],[]
with open('jeju_contents_tags.csv','r') as f:
    csv_reader = csv.reader(f)
    for line in csv_reader:
        if(line[1] == '0'):
            non_ads.append(line[0])
        elif(line[1] == '1'):
            ads.append(line[0])
        elif(line[1] == '2'):
            semi_ads.append(line[0])

In [355]:
#제주 여행과 관련이 없는 광고 - 주로 소재지만 제주인 경우 (ex. 제주 타투가게, 제주 옷가게, 제주 미용실...)
len(ads), ads[:3]

(86,
 ['사이즈가자꾸깨지는탓에 바로픽업이 힘들어져서 속상하네용  내일또 신상입고되니 많이찾아와주세요    2헨느  제주옷가게  아동의류  아동의류쇼핑몰  구남동옷가게  2reine  jeju  등원룩  문센룩  아동데일리룩  쥬니어의류쇼핑몰  링크클릭 ',
  '오늘 하루도 감사드려요 신상하러 못올라갔지만  거래처신상보면서 신상추가 하고있어요  사진으로만 보고 주문하다보니  핀이랑 머리띠가 좀많이들어왔지만 다 이쁘다고 해주셔서 감사해요 ',
  ' 우리아이 생일 키즈홈파티 대여 오픈 해요  첫번째 컨셉인 다이너소어파티 입니다  집에서도 다양한 컨셉으로 파티를 즐기실수 있도록준비해 드릴꺼에요  다빌리죠 상호명에 맞게 첫컨셉 잡은대로 찾아가는 중이에요  부족하지만 항상 예쁘다 해주셔서 감사해요   jeju 제주 다빌리죠 Dabillijyo 홈파티 제주카페 제주여행 제주파티 제주공항근처 제주돌스냅 제주웨딩스냅 제주파티샵 제주맘 제주아이 제주에서 제주한달살기 제주아기 제주라이프 제주커플 hbd 제주프로포즈 레터링풍선  제주풍선 제주파티소품  키즈파티 제주육아 헬륨풍선 꽃풍선 제주꽃풍선 제주이벤트'])

In [356]:
#광고가 아닌 모든 게시글 (ex. 여행 후기, 일상, 투어)
len(non_ads), non_ads[:3]

(569,
 ['누워서 한손으로  코야 잡아보기  부시럭부시럭  소리나서 놀랐더니유쨔니가 코야 만지며 놀고있더라구요 일어나면 조용히 놀고 있는 쪼꼬미틴구 ',
  ' 로즈데이 챙겨주는   ',
  ' Social Distancing   So In Gook Theme Park  2019 05 12 reminiscing  Jeju  tbt'])

In [357]:
#제주 여행과 관련이 있는 광고 (ex. 민박, 게스트하우스, 스쿠버다이빙, 카페)
len(semi_ads), semi_ads[:3]

(127,
 ['다이빙   뒷풀이 또한 완벽했던 우리들   다음주에 또 만난다는건 안비밀   율강사가 육지로 날아갈게요    제주 서귀포시 칠십리로 490 01079170079  01038074475 Kakaotalk  yul0079 율랜드   율강사  현강사  체험다이빙  율랜드스쿠버  oceantag  스쿠버교육  펀다이빙  보트다이빙  야간다이빙  섶섬  제주율랜드  jeju  jejuisland  제지기오름  보목포구  제주도가볼만한곳  서귀포스쿠버  오픈워터교육  제주도스킨스쿠버  제주도스쿠버다이빙  서귀포스쿠버교육 오산다이브리조트  신세계다이버  오산리이장님  밥식혜감사합니다',
  ' jeju  applemango  flake  빙수먹기  좋은날날씨가 습해서 빙수가 땡기는날100  제주 애플망고빙수 드시러 오세요  제주망고 애플망고 jeju mangoflake  villadeato     제주빙수 빙수스타그램 빙수맛집 빙수그램 맛있는 망고 달코롬 망고빙수 제주도 서귀포시 이중섭거리 올레시장 빙수   망빙 제주는 여름날씨 망고스무디 mango 이중섭거리 제주애플망고빙수',
  '게스트분들이랑 제주 포토스팟 갯깍주상절리대 다녀왔어요       소규모파티게하 레스고 소등없는게하 모두동의하에 노래방  보드게임  술게임 사정없이 하는게하 이렇게 놀다가 해뜨고 자도 난모룸  실제로 여행 못하신분들 많이봄  물론 써니도 하루가 사라지는 마법 혼자와도 우리는 위아더월드 귀찮을 정도로 말걸음 조식 원하시면해드림 써니표샌드위치와 갓뚜기라면중 택 1 제주도 요지 애월과 협재사이에 있는 게하 곽지해수욕장5분  협재 애월10분 만능버스 202번 타는 정류장 집앞 2분거리에 있음 공항에서 서귀포까지 다감 2박하시면 피크닉세트 빌려드림 번거롭게 짐챙길필요없이 써니가 다 준비해드림  서핑  패들보드 바다놀이기구등 원하시면 시중가보다 저렴하게 연결해드림 같이가자고 하시면 못이기는척 같이가드림 쥔장이 요리 조금 한다고하니 재료 사오시면 다 해드림 맛보장은 못ㅋ함ㅋ 나만맛있음 야외

### Tokenizing

In [358]:
from konlpy.tag import Okt

okt = Okt()

In [359]:
#마지막 5개씩은 테스트 데이터셋
ads_LCK, ads_Lth3 = [],[]
for i in range(80):
    for j in ads[i].split():
        ads_LCK+= okt.morphs(j)
        
for i in range(len(ads_LCK)):
    if len(ads_LCK[i]) >1:
        ads_Lth3.append(ads_LCK[i])
        
        
non_ads_LCK, non_ads_Lth3 = [],[]
for i in range(175):
    for j in non_ads[i].split():
        non_ads_LCK+= okt.morphs(j)
        
for i in range(len(non_ads_LCK)):
    if len(non_ads_LCK[i]) >1:
        non_ads_Lth3.append(non_ads_LCK[i])
        
        
semi_ads_LCK, semi_ads_Lth3 = [],[]
for i in range(60):
    for j in semi_ads[i].split():
        semi_ads_LCK+= okt.morphs(j)
        
for i in range(len(semi_ads_LCK)):
    if len(semi_ads_LCK[i]) >1:
        semi_ads_Lth3.append(semi_ads_LCK[i])

In [360]:
ads_Lth3[:6],non_ads_Lth3[:6],semi_ads_Lth3[:6]

(['사이즈', '자꾸', '깨지는', '바로', '픽업', '힘들어져서'],
 ['누워서', '한손으로', '코야', '잡아보기', '부시럭부시럭', '리나'],
 ['다이빙', '풀이', '또한', '완벽했던', '우리', '다음주'])

In [361]:
len(ads_Lth3),len(non_ads_Lth3),len(semi_ads_Lth3)

(3663, 3703, 3846)

### Probability

수업시간에 했던 방식과 동일한 방식으로 각 토큰의 확률을 구해보았다. 

In [362]:
ads_LCKF = pd.DataFrame(ads_Lth3, columns=['token'])
ads_c = pd.DataFrame(ads_LCKF.groupby(['token'])['token'].count()).rename(columns = {'token':'count'}).reset_index()
ads_c['add one'] = ads_c['count']+1
ads_c['Prob'] = ads_c['add one']/ads_c['add one'].sum()
ads_c['LN(P)'] = np.log(ads_c['Prob'])

non_ads_LCKF = pd.DataFrame(non_ads_Lth3, columns=['token'])
non_ads_c = pd.DataFrame(non_ads_LCKF.groupby(['token'])['token'].count()).rename(columns = {'token':'count'}).reset_index()
non_ads_c['add one'] = non_ads_c['count']+1
non_ads_c['Prob'] = non_ads_c['add one']/non_ads_c['add one'].sum()
non_ads_c['LN(P)'] = np.log(non_ads_c['Prob'])

semi_ads_LCKF = pd.DataFrame(semi_ads_Lth3, columns=['token'])
semi_ads_c = pd.DataFrame(semi_ads_LCKF.groupby(['token'])['token'].count()).rename(columns = {'token':'count'}).reset_index()
semi_ads_c['add one'] = semi_ads_c['count']+1
semi_ads_c['Prob'] = semi_ads_c['add one']/semi_ads_c['add one'].sum()
semi_ads_c['LN(P)'] = np.log(semi_ads_c['Prob'])

아래는 각 묶음별로 가장 출현빈도가 높은 단어들을 5개씩 나타낸 결과이다.
해시태그 jeju로 검색한 결과이기 때문에 제주, jeju, 제주도 등의 단어가 세 묶음 모두에게서 자주 발견됨을 관찰할 수 있다.
하지만 공통적으로 많이 나타나는 토큰은 세 집단을 구별하는데 큰 의미가 없기 때문에 이를 제거하여 동일한 과정을 다시 수행해보았다.

In [363]:
ads_c.sort_values(by=['add one'], ascending=False).iloc[:5]

,token,count,add one,Prob,LN(P)
1656,제주,138,139,0.025063,-3.686358
1120,네일,56,57,0.010278,-4.577781
288,jeju,54,55,0.009917,-4.613499
1657,제주도,46,47,0.008475,-4.770685
354,ng,38,39,0.007032,-4.957271


In [364]:
non_ads_c.sort_values(by=['add one'], ascending=False).iloc[:5]

,token,count,add one,Prob,LN(P)
1860,제주,117,118,0.020268,-3.898714
304,jeju,98,99,0.017004,-4.074279
1654,여행,75,76,0.013054,-4.338666
1861,제주도,68,69,0.011852,-4.435293
1084,그램,48,49,0.008416,-4.777579


In [365]:
semi_ads_c.sort_values(by=['add one'], ascending=False).iloc[:5]

,token,count,add one,Prob,LN(P)
1512,제주,207,208,0.037057,-3.295303
1039,맛집,154,155,0.027614,-3.589415
1513,제주도,67,68,0.012115,-4.413333
1293,애월,57,58,0.010333,-4.572398
166,jeju,43,44,0.007839,-4.848651


### Probability - removed common words

In [366]:
common_words = ["제주","제주도","jeju","제주"]

new_ads_Lth3,new_non_ads_Lth3,new_semi_ads_Lth3 = [],[],[]
for i in ads_Lth3:
    if i not in common_words:
        new_ads_Lth3.append(i)
        
for i in non_ads_Lth3:
    if i not in common_words:
        new_non_ads_Lth3.append(i)
        
for i in semi_ads_Lth3:
    if i not in common_words:
        new_semi_ads_Lth3.append(i)

In [367]:
len(ads_Lth3),len(new_ads_Lth3)

(3663, 3409)

In [368]:
len(non_ads_Lth3),len(new_non_ads_Lth3)

(3703, 3401)

In [369]:
len(semi_ads_Lth3),len(new_semi_ads_Lth3)

(3846, 3519)

In [370]:
ads_LCKF = pd.DataFrame(new_ads_Lth3, columns=['token'])
ads_c = pd.DataFrame(ads_LCKF.groupby(['token'])['token'].count()).rename(columns = {'token':'count'}).reset_index()
ads_c['add one'] = ads_c['count']+1
ads_c['Prob'] = ads_c['add one']/ads_c['add one'].sum()
ads_c['LN(P)'] = np.log(ads_c['Prob'])

non_ads_LCKF = pd.DataFrame(new_non_ads_Lth3, columns=['token'])
non_ads_c = pd.DataFrame(non_ads_LCKF.groupby(['token'])['token'].count()).rename(columns = {'token':'count'}).reset_index()
non_ads_c['add one'] = non_ads_c['count']+1
non_ads_c['Prob'] = non_ads_c['add one']/non_ads_c['add one'].sum()
non_ads_c['LN(P)'] = np.log(non_ads_c['Prob'])

semi_ads_LCKF = pd.DataFrame(new_semi_ads_Lth3, columns=['token'])
semi_ads_c = pd.DataFrame(semi_ads_LCKF.groupby(['token'])['token'].count()).rename(columns = {'token':'count'}).reset_index()
semi_ads_c['add one'] = semi_ads_c['count']+1
semi_ads_c['Prob'] = semi_ads_c['add one']/semi_ads_c['add one'].sum()
semi_ads_c['LN(P)'] = np.log(semi_ads_c['Prob'])

In [377]:
ads_c.sort_values(by=['add one'], ascending=False).iloc[:5]

,token,count,add one,Prob,LN(P)
1118,네일,56,57,0.010779,-4.530144
353,ng,38,39,0.007375,-4.909634
1720,청년,31,32,0.006051,-5.107459
1411,스냅,31,32,0.006051,-5.107459
1543,웨딩,23,24,0.004539,-5.395142


In [378]:
non_ads_c.sort_values(by=['add one'], ascending=False).iloc[:5]

,token,count,add one,Prob,LN(P)
1652,여행,75,76,0.013778,-4.284675
1082,그램,48,49,0.008883,-4.723588
1515,스타,39,40,0.007252,-4.926529
1795,일상,31,32,0.005801,-5.149672
197,daily,26,27,0.004895,-5.319571


In [379]:
semi_ads_c.sort_values(by=['add one'], ascending=False).iloc[:5]

,token,count,add one,Prob,LN(P)
1037,맛집,154,155,0.029345,-3.528635
1291,애월,57,58,0.010981,-4.511617
1340,여행,38,39,0.007384,-4.908498
1606,카페,29,30,0.005680,-5.170863
1590,추천,29,30,0.005680,-5.170863


### Test

In [374]:
Test_data = ads[-5:]+non_ads[-5:]+semi_ads[-5:]

Test_data_split = []
for i in range(len(Test_data)):
    tmp=[]
    for j in Test_data[i].split():
        tmp += okt.morphs(j)
    Test_data_split.append(tmp)

In [375]:
ads_prob = [[] for cols in range(len(Test_data_split))]
for i in range(len(Test_data_split)):
    for j in range(len(Test_data_split[i])):
        if len(Test_data_split[i][j]) > 1 and j not in common_words:
            if(ads_c[ads_c['token'].isin([Test_data_split[i][j]])]['LN(P)'].empty):
                ads_prob[i].append(math.log(1/ads_c['add one'].sum()))
            else:
                ads_prob[i].append(ads_c[ads_c['token'].isin([Test_data_split[i][j]])]['LN(P)'].values[0])
        else:
            ads_prob[i].append(0.0)

non_ads_prob = [[] for cols in range(len(Test_data_split))]
for i in range(len(Test_data_split)):
    for j in range(len(Test_data_split[i])):
        if len(Test_data_split[i][j]) > 1:
            if(non_ads_c[non_ads_c['token'].isin([Test_data_split[i][j]])]['LN(P)'].empty):
                non_ads_prob[i].append(math.log(1/non_ads_c['add one'].sum()))
            else:
                non_ads_prob[i].append(non_ads_c[non_ads_c['token'].isin([Test_data_split[i][j]])]['LN(P)'].values[0])
        else:
            non_ads_prob[i].append(0.0)
            
semi_ads_prob = [[] for cols in range(len(Test_data_split))]
for i in range(len(Test_data_split)):
    for j in range(len(Test_data_split[i])):
        if len(Test_data_split[i][j]) > 1:
            if(semi_ads_c[semi_ads_c['token'].isin([Test_data_split[i][j]])]['LN(P)'].empty):
                semi_ads_prob[i].append(math.log(1/semi_ads_c['add one'].sum()))
            else:
                semi_ads_prob[i].append(semi_ads_c[semi_ads_c['token'].isin([Test_data_split[i][j]])]['LN(P)'].values[0])
        else:
            semi_ads_prob[i].append(0.0)

### Result

In [376]:
result = []
for i in range(len(ads_prob)):
    p_ads,p_non_ads,p_semi_ads = np.sum(ads_prob[i]),np.sum(non_ads_prob[i]),np.sum(semi_ads_prob[i])
#     print(p_ads,p_non_ads,p_semi_ads)
    p_max = max([p_ads,p_non_ads,p_semi_ads])
    if(p_max == p_ads):
        print("Ads")
    elif(p_max == p_non_ads):
        print("Non-ads")
    else:
        print("Semi-ads")

Semi-ads
Semi-ads
Semi-ads
Ads
Ads
Non-ads
Non-ads
Semi-ads
Non-ads
Non-ads
Semi-ads
Semi-ads
Semi-ads
Semi-ads
Semi-ads
